In [1]:
import hues
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, CrossEntropyLoss

In [2]:
DATA_DIR = '../05-Transforms/data/'

In [3]:
transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_data = torchvision.datasets.CIFAR10(
    root=DATA_DIR,
    transform=transformer,
    train=True,
    download=True
)

test_data = torchvision.datasets.CIFAR10(
    root=DATA_DIR,
    transform=transformer,
    train=False,
    download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_data), len(test_data)

(50000, 10000)

In [5]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

![](https://i.imgur.com/rCyvEOB.png)

In [6]:
# 搭建神经网络
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()

        self.model = torch.nn.Sequential(
            Conv2d(3, 32, 5, padding=2),  # Input: 3@32×32, Output: 32@32×32
            MaxPool2d(2),  # Input: 32@32×32, Output: 32@16×16
            Conv2d(32, 32, 5, padding=2),  # Input: 32@16×16, Output: 32@16×16
            MaxPool2d(2),  # Input: 32@16×16, Output: 32@8×8
            Conv2d(32, 64, 5, padding=2),  # Input: 32@8×8, Output: 64@8×8
            MaxPool2d(2),  # Input: 64@8×8, Output: 64@4×4
            Flatten(),  # Input: 64@4×4, Output: 1024
            Linear(1024, 64),  # Input: 1024, Output: 64
            Linear(64, 10)  # Input: 64, Output: 10
        )

    def forward(self, inputs):
        outputs = self.model(inputs)
        return outputs

In [7]:
# 创建网络模型
module = MyModule()

# 在GPU上训练模型
if torch.cuda.is_available():
    hues.info('GPU is available in this computer.')
    module.cuda()

# 第二种调用方式
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# module.to(device)

print(module)

09:53:18 - INFO - GPU is available in this computer.
MyModule(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [8]:
# 损失函数
loss_fun = CrossEntropyLoss()
if torch.cuda.is_available():
    # 转移到GPU上
    loss_fun.cuda()

# 优化器
learning_rate = 0.0001
optimizer = torch.optim.SGD(module.parameters(), lr=learning_rate)

# 使用tensorboard可视化网络结构
writer = SummaryWriter('./logs')

In [9]:
for epoch in range(30):
    epoch_train_loss = 0  # 对每一轮的loss进行累加，观察经过多轮学习后，loss是否有下降
    epoch_test_loss = 0
    pred_right_num = 0  # 测试集中预测正确的数量

    # 对训练集的每一个批量数据进行训练
    # module.train()  # 这一行代码可有可无，只针对特定模型需要设置，官方文档：https://blog.csdn.net/u014764291/article/details/105924182
    for i, data in enumerate(train_loader):
        imgs, targets = data

        # 将数据转移到GPU
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()

        outputs = module(imgs)
        res_loss = loss_fun(outputs, targets)
        optimizer.zero_grad()
        res_loss.backward()
        optimizer.step()
        epoch_train_loss += res_loss
        # if i % 100 == 0:
        #     print(f'第{epoch}轮中的第{i}批数据训练Loss值为：{res_loss.item()}.')

    # 检验当前模型在测试集上的效果
    # module.eval()  # 参考module.train()
    with torch.no_grad():  # 在测试集上检验效果，不需要进行梯度下降优化
        for i, data in enumerate(test_loader):
            imgs, targets = data

            # 将数据转移到GPU
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()

            outputs = module(imgs)
            res_loss = loss_fun(outputs, targets)
            epoch_test_loss += res_loss

            item_train_num = epoch * len(data) + i + 1

            pred_right_num += (outputs.argmax(1) == targets).sum()

    writer.add_scalar('Train Loss', epoch_train_loss.item(), epoch)
    writer.add_scalar('Test Loss', epoch_test_loss.item(), epoch)
    writer.add_scalar('Accuracy on Test Dataset', pred_right_num / len(test_data), epoch)

    torch.save(module, f'./module/module_{epoch}.pth')

    hues.info(
        f'第{epoch}轮的训练集累积Loss值为：{epoch_train_loss.item()}，测试集累积Loss值为：{epoch_test_loss.item()}，测试集上的预测正确率为{pred_right_num / len(test_data)}。')

# tensorboard --logdir='logs'

09:53:35 - INFO - 第0轮的训练集累积Loss值为：1800.701171875，测试集累积Loss值为：359.6269226074219，测试集上的预测正确率为0.09879999607801437。
09:53:49 - INFO - 第1轮的训练集累积Loss值为：1800.0496826171875，测试集累积Loss值为：359.4991149902344，测试集上的预测正确率为0.09829999506473541。
09:54:03 - INFO - 第2轮的训练集累积Loss值为：1799.420654296875，测试集累积Loss值为：359.3764953613281，测试集上的预测正确率为0.09749999642372131。
09:54:16 - INFO - 第3轮的训练集累积Loss值为：1798.8453369140625，测试集累积Loss值为：359.2574768066406，测试集上的预测正确率为0.09559999406337738。
09:54:29 - INFO - 第4轮的训练集累积Loss值为：1798.277587890625，测试集累积Loss值为：359.1410217285156，测试集上的预测正确率为0.09369999915361404。
09:54:42 - INFO - 第5轮的训练集累积Loss值为：1797.7236328125，测试集累积Loss值为：359.0340881347656，测试集上的预测正确率为0.09510000050067902。
09:54:54 - INFO - 第6轮的训练集累积Loss值为：1797.168212890625，测试集累积Loss值为：358.9263000488281，测试集上的预测正确率为0.09839999675750732。
09:55:07 - INFO - 第7轮的训练集累积Loss值为：1796.6322021484375，测试集累积Loss值为：358.8101806640625，测试集上的预测正确率为0.10269999504089355。
09:55:19 - INFO - 第8轮的训练集累积Loss值为：1796.08349609375，测试集累积Loss值为：358.7056884765625，测试集上的预测正确

In [7]:
print('-' * 100, f'Epoch {1}')

---------------------------------------------------------------------------------------------------- Epoch 1


In [6]:
import numpy as np

X = np.zeros((8, 8))
X

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
links = [
    (0, 1), (0, 2), (0, 7),
    (1, 0), (1, 2), (1, 3), (1, 4), (1, 7),
    (2, 0), (2, 1), (2, 3),
    (3, 1), (3, 2), (3, 4), (3, 5), (3, 7),
    (4, 1), (4, 3), (4, 5), (4, 6), (4, 7),
    (5, 3), (5, 4), (5, 6),
    (6, 4), (6, 5),
    (7, 0), (7, 1), (7, 3), (7, 4)
]

In [9]:
for source, target in links:
    X[source, target] = 1

In [10]:
X

array([[0., 1., 1., 0., 0., 0., 0., 1.],
       [1., 0., 1., 1., 1., 0., 0., 1.],
       [1., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 1., 0., 1.],
       [0., 1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [1., 1., 0., 1., 1., 0., 0., 0.]])

In [11]:
for i in range(len(X)):
    for j in range(i, len(X)):
        if X[i, j] != X[j, i]:
            hues.error(f'ERROR, {i, j}')

In [12]:
#求矩阵特征值以及特征向量
eig_value,eig_vector=np.linalg.eig(X)
eig_value

array([ 4.05972216e+00,  1.79634500e+00, -2.19294057e+00, -1.74979075e+00,
       -1.32142784e+00, -7.95801735e-01,  2.13857093e-16,  2.03893733e-01])

In [13]:
eig_vector

array([[ 2.80844511e-01,  4.15910407e-01, -4.37649280e-01,
        -1.43312092e-01, -1.63468950e-01,  4.19050065e-01,
        -5.77350269e-01, -2.02795000e-02],
       [ 4.53229578e-01,  2.63750238e-01,  1.20985989e-01,
         4.94574625e-01, -4.99265675e-01, -4.62185086e-01,
        -1.25356763e-16,  5.38128644e-02],
       [ 2.91254960e-01,  3.41876413e-01,  3.90458933e-01,
        -3.16399432e-01,  2.95429059e-01, -1.51618909e-01,
        -3.86688574e-16, -6.59892162e-01],
       [ 4.48340127e-01, -6.55326615e-02, -5.39589945e-01,
         2.02370265e-01,  2.72346440e-01,  1.63793612e-01,
         5.77350269e-01, -1.68081240e-01],
       [ 4.23880416e-01, -3.59959666e-01, -1.26828731e-01,
        -6.80651823e-01, -1.64412023e-01, -3.43740301e-01,
         3.72344829e-16,  2.57280573e-01],
       [ 2.56105250e-01, -5.04878182e-01,  3.50378552e-01,
         7.57800326e-02, -4.11486567e-01,  5.46873829e-01,
        -3.51498730e-16, -2.87416422e-01],
       [ 1.67495616e-01, -4.814430